In [55]:
import jieba
import jieba.posseg as pseg
import os
import re
import sys
import time
from sklearn.decomposition import PCA
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pymysql
 
corpus=[]

pattern = re.compile("[\u4e00-\u9fa5]+")
domain_ist = []


In [56]:
for maindir, subdir, file_name_list in os.walk('C:/Users/12045/OneDrive/桌面/homepage'):
    for file_name in file_name_list:
        file_path = 'C:/Users/12045/OneDrive/桌面/homepage/' + file_name
        domain_ist.append(file_name[:-5])
        with open(file_path, encoding='utf-8') as f:
            html = f.read()
            content = ''
            for text in pattern.finditer(html):
                content += text.group(0) + ' '
                # print(text.group(0))
            corpus.append(content)

In [57]:
print(len(corpus))

12206


In [58]:
vectorizer=CountVectorizer(max_features=50)#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值
tfidf=transformer.fit_transform(vectorizer.fit_transform(corpus))#是计算tf-idf，将文本转为词频矩阵

In [60]:
conn = pymysql.Connect(  # 配置数据库
    host='1.15.220.155',
    port=3306,
    user='test',
    password='991125',
    db='spider',
    charset='UTF8'
)
cursor = conn.cursor()

word=vectorizer.get_feature_names()#获取词袋模型中的所有词语
weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
sql_update = 'update cluster set tfidf_sequence = %s where domain = %s'
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for遍历某一类文本下的词语权重
    sequence = ''
    for j in range(len(word)):
        first = str(weight[i][j]).split('.')[1][0]
        sequence += first
    cursor.execute(sql_update, [sequence, domain_ist[i]])
    conn.commit()
    
    

C:\Users\12045\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# word=vectorizer.get_feature_names()#获取词袋模型中的所有词语
# weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
# print( '原有维度: ', len(weight[0]))
# pca = PCA(n_components=10) # 初始化PCA
# X = pca.fit_transform(weight) # 返回降维后的数据
# print( '降维后维度: ', len(X[0]))
# print(len(X))

原有维度:  60
降维后维度:  10
12206


C:\Users\12045\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# for i in range(len(X)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for遍历某一类文本下的词语权重
#     print ("-------这里输出第",i,"类文本的词语tf-idf权重------")
#     for j in range(len(X[0])):
#         print(X[i][j])
#     print('***')

-------这里输出第 0 类文本的词语tf-idf权重------
0.5263841157965007
0.1769249525481206
-0.12400552836609893
0.08398462552564764
-0.2152591461257295
0.3682942236107657
0.022608008912993062
-0.10921964700901916
0.045653775943304054
0.2371711192205783
***
-------这里输出第 1 类文本的词语tf-idf权重------
-0.43908453740458964
0.03398896998977673
0.1275126035861361
-0.09571368113752395
-0.04623187269521539
-0.09272385848776804
0.09425885603913567
0.017583323860047247
0.06478993726339159
0.07606535700392973
***
-------这里输出第 2 类文本的词语tf-idf权重------
-0.4387838859575767
0.03183488155187181
-0.02174987449132682
0.08741731214481901
0.030311865712984016
-0.10838178933142892
0.006468106682916841
-0.05598677299439057
0.06673467812657738
-0.11889122454519704
***
-------这里输出第 3 类文本的词语tf-idf权重------
0.31837896625480444
-0.21282526000555207
-0.18245969655556077
0.12833142355040736
-0.06532084566220223
0.6462081349493872
-0.015530078860954062
0.34722144615572925
-0.3398679134402624
0.03184300353552635
***
-------这里输出第 4 类文本的词语tf-id

KeyboardInterrupt: 